## Inference guide

In [1]:
import os
import shutil
import os.path as osp

import json 
import os


In [2]:
TEST_IMG_DIR = '../data/public/img/test/'

In [3]:
from natsort import natsorted

In [4]:
# current_dir = os.getcwd()
# current_dir


In [5]:
# test_img_size = 380
test_img_size = None

In [6]:
runs_dir = '/home/tuantran/ucc-ai-quest-2023/runs/equal_ensemble_public'

In [7]:
exp_list = [x for x in os.listdir(runs_dir) if osp.isdir(osp.join(runs_dir, x))]
exp_list

['fold_3-regnety_upernet-epoch_70',
 'mask2former-ade-small-epoch_28',
 'fold_0-segformer-b3-ade-epoch_85',
 'fold_3-mask2former-ade-Mask2former-epoch_97',
 'fold_4-mask2former-ade-small-epoch_93',
 'fold_2-convnext_sfnet-color_augment-smooth_losses-epoch_13',
 'fold_2-regnety_upernet-epoch_94',
 'convnext_sfnet-adamw-color_augment-smooth_losses-RepeatedWarmupPolyLR-epoch_93',
 'fold_0-mask2former-ade-epoch_8',
 'fold_1-regnety_upernet-epoch_99',
 'fold_4-segformer-b3-ade-epoch_99',
 'fold_3-mask2former-city_21k-large-epoch_89',
 'fold_0-mask2former-ade-small-epoch_82',
 'fold_2-segformer-b4-ade-SIZE-380-epoch_67',
 'fold_4-convnext_ssl_sfnet-epoch_55',
 'fold_0-convnext_ssl_sfnet-epoch_98',
 'fold_1-segformer-b4-ade-SIZE-380-epoch_77',
 'fold_1-mask2former-ade-epoch_6',
 'fold_0-regnety_upernet-epoch_91',
 'fold_2-segformer-b4-ade-epoch_91',
 'fold_2-mask2former-ade-epoch_98',
 'mask2former-ade-epoch_31',
 'fold_0-segformer-b5-ade-epoch_83',
 'fold_3-segformer-b5-ade-380-epoch_55',
 '

In [8]:
for EXP_NAME in natsorted(exp_list):
    exp_dir = osp.join(runs_dir, EXP_NAME)
    CFG_PATH = osp.join(exp_dir, 'inference.yml')
    
    # shutil.copy(CFG_PATH, EXP_NAME)
    # with open(CFG_PATH, 'w') as f:
    #     f.write(config)
    
    if test_img_size is not None:
        with open(CFG_PATH, 'r') as f:
            config = f.read()
            config = config.replace('SIZE: null # ORIGINAL SIZE', f'SIZE: {test_img_size}')
            config = config.replace('SIZE: 380', f'SIZE: {test_img_size}')
            config = config.replace('SIZE: 384', f'SIZE: {test_img_size}')
        
        with open(CFG_PATH, 'w') as f:
            f.write(config)
            
        with open(CFG_PATH, 'r') as f:
            config = f.read()
            assert f'SIZE: {test_img_size}' in config
    
    RUN_FLAG = True
    with open(osp.join(exp_dir, 'results.json'), 'r') as f:
        j = json.loads(f.read())
        if len(j) == len(os.listdir(TEST_IMG_DIR)):
            RUN_FLAG = False

    if RUN_FLAG:
        print(f'Running experiment:', EXP_NAME)
        pretrained_weights = osp.join(exp_dir, 'best.ckpt')
        !ucc-pred   -c  {exp_dir}/inference.yml \
                -o  global.pretrained={pretrained_weights}\
                    data.IMG_DIR={TEST_IMG_DIR} \
                    global.save_path={exp_dir}/results.json
                    
        with open(osp.join(exp_dir, 'results.json'), 'r') as f:
            j = json.loads(f.read())
        assert len(j) == len(os.listdir(TEST_IMG_DIR))
        
        !rm -rf {exp_dir}/results.zip
        !zip {exp_dir}/results.zip {exp_dir}/results.json
    else:
        print('Skipping:', EXP_NAME)

Skipping: convnext_sfnet-adamw-color_augment-smooth_losses-RepeatedWarmupPolyLR-epoch_93
Skipping: convnext_v1_sfnet-adamw-color_augment-smooth_losses-epoch_27
Skipping: fold_0-convnext_sfnet-color_augment-smooth_losses-epoch_90
Skipping: fold_0-convnext_ssl_sfnet-epoch_98
Skipping: fold_0-mask2former-ade-21k-epoch_98
Skipping: fold_0-mask2former-ade-epoch_8
Skipping: fold_0-mask2former-ade-small-epoch_82
Skipping: fold_0-regnety_upernet-epoch_91
Skipping: fold_0-segformer-b3-ade-epoch_85
Skipping: fold_0-segformer-b5-ade-380-epoch_88
Skipping: fold_0-segformer-b5-ade-epoch_83
Skipping: fold_1-convnext_ssl_sfnet-epoch_96
Skipping: fold_1-mask2former-ade-epoch_6
Skipping: fold_1-regnety_upernet-epoch_99
Skipping: fold_1-segformer-b4-ade-SIZE-380-epoch_77
Skipping: fold_1-segformer-b4-ade-epoch_97
Skipping: fold_2-convnext_sfnet-color_augment-smooth_losses-epoch_13
Skipping: fold_2-mask2former-ade-epoch_98
Skipping: fold_2-regnety_upernet-epoch_94
Skipping: fold_2-segformer-b4-ade-SIZE-3